In [ ]:
import h5py
import pandas as pd

In [ ]:
quotes = pd.read_hdf("data/quotes.h5")

In [ ]:
ed_quotes = quotes[quotes['alignement_politique'] == 'extremegauche_gaucheradicale'].copy()

# Cues
Beaucoup de valeurs manquantes, et pas l'impression d'évolutions notables au cours du temps ?

In [ ]:
ed_clean = ed_quotes[
    ed_quotes['cue_lemma'].notna() & (ed_quotes['cue_lemma'] != 'nan')
].copy()
ed_clean['year'] = ed_clean['date'].dt.year

lemma_counts = (
    ed_clean
    .groupby(['year', 'cue_lemma'])
    .size()
    .reset_index(name='count')
)

top_lemmas_per_year = (
    lemma_counts
    .sort_values(['year', 'count'], ascending=[True, False])
    .groupby('year')
    .head(5)
).reset_index(drop=True)

top_lemmas_per_year.style.hide(axis="index")

In [ ]:
quotes_clean = quotes[
    quotes['cue_lemma'].notna() & (quotes['cue_lemma'] != 'nan')
].copy()
quotes_clean['year'] = quotes_clean['date'].dt.year

lemma_counts = (
    quotes_clean
    .groupby(['alignement_politique', 'cue_lemma'])
    .size()
    .reset_index(name='count')
)

lemma_counts['rank'] = lemma_counts.groupby('alignement_politique')['count'].rank(method='first', ascending=False)
lemma_counts = lemma_counts[lemma_counts['rank'] <= 15]
pivot_table = lemma_counts.pivot(index='alignement_politique', columns='rank', values='cue_lemma')
pivot_table.columns = [f"Rank {int(col)}" for col in pivot_table.columns]
pivot_table.style